In [1]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import wandb
import copy
import time
from datetime import datetime
from pytz import timezone
import seaborn as sns

import cv2

from torchvision.transforms import ToTensor,Compose,ColorJitter,RandomRotation,RandomHorizontalFlip,Resize,CenterCrop,Normalize,GaussianBlur,RandomCrop

import timm

import torch
import torch.nn as nn
import torch.optim as optm
import torch.nn.functional as F
import torch.distributions as dist
from torch.utils.data import Dataset, DataLoader

import torchvision

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib import gridspec

from torchinfo import summary
from tqdm.auto import tqdm

%matplotlib inline
device = 'cuda' if torch.cuda.is_available() else 'cpu'
cwd=os.path.dirname(os.getcwd())
KST=timezone('Asia/Seoul')
date=str(datetime.now().astimezone(KST))[:10]

In [2]:
#프로젝트 이름 설정과 저장경로
project_name='BestEachClassEfficientNetTest'

In [3]:
#Randomness 제어
random_seed=2021
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(random_seed)
random.seed(random_seed)

In [4]:
transform={'trans':Compose([
    Resize((256*512//384,256)),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),'regtrans':Compose([
    Resize((256*512//384,256)),
    GaussianBlur(3),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    RandomCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]),'uptrans':Compose([
    Resize((256*512//384,256)),
    RandomHorizontalFlip(),
    RandomRotation(20),
    CenterCrop(224),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])}

def make_images(meta,img_dir,train):
    images=[]
    labels=[]
    ages=[]
    data=pd.DataFrame({'images':[],'labels':[]})
    if train:
        for idx in range(len(meta)):
            folder_path=os.path.join(img_dir, meta.path.iloc[idx])
            for img in os.listdir(folder_path):
                if '._' in img:
                    continue
                images.append(Image.open(os.path.join(folder_path,img)))
                labels.append((('incorrect' in img)+('normal' in img)*2)*6+(meta.gender.iloc[idx]=='female')*3+(30<=meta.age.iloc[idx])+(60<=meta.age.iloc[idx]))
                ages.append(meta.age.iloc[idx])
    else:
        for img_id in meta.ImageID:
            images.append(Image.open(os.path.join(img_dir, img_id)))
        labels=[0 for _ in range(len(images))]
    data['images']=images
    data['labels']=labels
    return data

class ImageDataset(Dataset):
    def __init__(self,transform=transform,mod='train'):
        self.mod=mod
        self.train=mod!='test'
        self.md=['info','train']
        self.path=[os.path.join(cwd,'input/data/eval'),os.path.join(cwd,'input/data/train')]
        self.meta=pd.read_csv(os.path.join(self.path[self.train], f'{self.md[self.train]}.csv'))
        self.img_dir=os.path.join(self.path[self.train],'images')
        self.classes=[('Wear','Incorrect','Not Wear'),('남','여'),('<30','>=30 and <60','>=60')]
        self.trans=transform
        
        if self.train:
            self.criteria=1*(self.meta['age']>=30)+1*(self.meta['age']>=60)+3*(self.meta['gender']=='female')
            self.train_data,self.val_data=train_test_split(self.meta,test_size=0.2,stratify=self.criteria)
            self.train_data=make_images(self.train_data,self.img_dir,self.train)
            self.val_data=make_images(self.val_data,self.img_dir,self.train)
        else:
            self.data=make_images(self.meta,self.img_dir,self.train)
    
    def __len__(self):
        if self.mod=='train':
            l=len(self.train_data)
        elif self.mod=='valid':
            l=len(self.val_data)
        else:
            l=len(self.data)
        return  l
        
    
    def __getitem__(self, idx):
        if self.mod=='valid':
            images=self.val_data['images']
            labels=self.val_data['labels']
        elif self.mod=='train':
            images=self.train_data['images']
            labels=self.train_data['labels']
        else:
            images=self.data['images']
            labels=self.data['labels']
            

        label=labels.iloc[idx]
        image=images.iloc[idx]
        
        if self.mod=='train':
            if True:
                image=self.trans['uptrans'](image)
                return image,label
            else:
                image=self.trans['regtrans'](image)
                return image,label
        else:
            image=self.trans['trans'](image)
            return image,label
        return image,0,0

def conf_mat(y_true,y_pred):
    cm=confusion_matrix(y_true,y_pred)
    norm_cm=cm/np.sum(cm, axis=1)[:,None]
    indices=['wear,m,<30','wear,m,mask<>','wear,m,mask60','wear,f,<30','wear,f,<>','wear,f,60','inc,m,<30','inc,m,<>','inc,m,60','inc,f,<30','inc,f,<>','inc,f,60','nom,m,<30','nom,m,<>','nom,m,60','nom,f,<30','nom,f,<>','nom,f,60']
    cm=pd.DataFrame(norm_cm,index=indices,columns=indices)
    fig=plt.figure(figsize=(11,9))
    sns.heatmap(cm,annot=True)
    return fig


class SkewNormal(dist.normal.Normal):
    def __init__(self,mean=0,std=1,skew=0):
        super().__init__(loc=mean,scale=std)
        self.skew=skew
    def skewcdf(self,x):
        return 0.5 * (1 + torch.erf(self.skew*(x - self.loc) * self.scale.reciprocal() / np.sqrt(2)))

    def log_p(self,x):
        return self.log_prob(x)+torch.log(self.skewcdf(x))-torch.log(torch.tensor(2.))

def label_smoothing(label):
    label-=18
    label_0=SkewNormal(11.,7,-20).log_p
    label_11=SkewNormal(12.,7,20).log_p
    label_12=SkewNormal(41,7,-20).log_p
    label_2=dist.Normal(43.7,.65).log_prob
    return torch.exp(torch.vstack((label_0(label),torch.log(torch.exp(label_11(label))+torch.exp(label_12(label))),label_2(label))).T)

# 합친 모델 학습과 결과저장

In [5]:
net_mask = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)
net_gender = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=2)
net_age = timm.create_model('efficientnet_b3a', pretrained=True, num_classes=3)

#부분 모델 불러오기
checkpoint_dir=os.path.join('.',f'saved/{project_name}/checkpoint')
model_id=sorted([i for i in os.listdir(checkpoint_dir) if 'mask' in i])[-1]
checkpoint=torch.load(os.path.join(checkpoint_dir,model_id))
net_mask.load_state_dict(checkpoint['model_state_dict'])
model_id=sorted([i for i in os.listdir(checkpoint_dir) if 'gender' in i])[-1]
checkpoint=torch.load(os.path.join(checkpoint_dir,model_id))
net_gender.load_state_dict(checkpoint['model_state_dict'])
model_id=sorted([i for i in os.listdir(checkpoint_dir) if ('age' in i) and ('tensor' not in i)])[-1]
checkpoint=torch.load(os.path.join(checkpoint_dir,model_id))
net_age.load_state_dict(checkpoint['model_state_dict'])

net_mask.to(device)
net_gender.to(device)
net_age.to(device)

print('done')

done


In [6]:
#모델 구축
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.mask=net_mask
        self.gender=net_gender
        self.age=net_age
        
    def forward(self,x):
        mask=(nn.Softmax(dim=1)(self.mask(x))).view(x.size(0),3,1,1)
        gender=(nn.Softmax(dim=1)(self.gender(x))).view(x.size(0),1,2,1)
        age=(nn.Softmax(dim=1)(self.age(x))).view(x.size(0),1,1,3)
        return (mask*gender*age).view(x.size(0),18)

In [7]:
#모델 설정하기
model=MyModel()
model.to(device)
optim=optm.Adam(model.parameters())
scheduler = optm.lr_scheduler.StepLR(optim, step_size=1, gamma=0.75)

In [8]:
#학습 설정하기
num_epochs=100
batch_size=60
learning_rate=0.000001
best_f1=0.
stopper=0
weight=(torch.tensor([7/15,7/3,7/3]).view(3,1,1)*torch.tensor([2700/(2*1042),2700/(2*1658)]).view(1,2,1)*torch.tensor([900/1281*12,900/1227*15,900/192]).view(1,1,3)).view(-1).to(device)
loss_fn=nn.CrossEntropyLoss(weight=weight)
config={'epochs':num_epochs,'transform':'uptrans','batch_size':batch_size,'learning_rate':learning_rate, 'weight':weight,'f1_avg':'weighted'}

In [10]:
now='Runned'+'_'.join(str(datetime.now().astimezone(KST))[10:16].split(':'))
base_dir=os.path.join('.',f'saved/{project_name}/{date}/{now}')
wandb_dir=base_dir

dirs={'model':'','checkpoint':'','result':''}
for pth in ['model','checkpoint','result']:
    dirs[pth]=os.path.join(base_dir,pth)
    if not os.path.isdir(dirs[pth]):
        os.makedirs(dirs[pth])


wandb.init(project=project_name,config=config,dir=wandb_dir)
best_f1=0.
best_epoch=0
stopper=0
for e in range(num_epochs):   
    for mod in ['valid','test','train',]:
        print('mod:',mod)
        data=ImageDataset(mod=mod)

        if mod=='train':
            dataloader=DataLoader(data,batch_size=batch_size,shuffle=True,pin_memory=True)
            model.train()
        else:
            submission = pd.read_csv(os.path.join(data.path[0], 'info.csv'))
            dataloader=DataLoader(data,shuffle=False)
            model.eval()

        with tqdm(dataloader) as pbar:
            running_loss=0.
            running_acc=0.
            tot_pred=torch.tensor([]).to(device)
            tot_label=torch.tensor([]).to(device)
            cm_pred=torch.tensor([]).to(device)
            cm_label=torch.tensor([]).to(device)
            for n,(image,label) in enumerate(pbar):
                image=image.to(device)
                label=label.to(device)
                
                logit=model(image)
                _,pred=torch.max(logit,1)
                
                if data.train:
                    loss=loss_fn(logit,label)
                    running_loss+=loss.item()*image.size(0)
                    running_acc+=torch.sum(pred==label)/image.size(0)

                if mod=='train':
                    optim.zero_grad()
                    loss.backward()
                    optim.step()                    

                tot_pred=torch.hstack((tot_pred,pred))
                tot_label=torch.hstack((tot_label,label))
                if data.train:
                    pbar.set_postfix({'epoch' : e, 'loss' : running_loss/(n+1), 'accuracy' : float(running_acc)/(n+1),'F1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')})
                    if mod=='train':
                        wandb.log({f'{str(e).zfill(3)} running loss' : running_loss/(n+1), f'{str(e).zfill(3)} running accuracy' : float(running_acc)/(n+1),f'{str(e).zfill(3)} running f1 score':f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')})
            if data.train:
                epoch_loss=running_loss/len(dataloader.dataset)
                epoch_acc=running_acc*image.size(0)/len(dataloader.dataset)
                epoch_f1=f1_score(tot_label.cpu(),tot_pred.cpu(),average='weighted')
                print(f"현재 epoch-{e}의 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}, F1 score : {epoch_f1:.3f}")
            elif mod=='test':
                submission['ans'] = tot_pred.cpu().numpy().astype('int64')
                submission.to_csv(os.path.join(dirs['result'], f'submission_{project_name}_{str(e).zfill(3)}_{epoch_f1}.csv'), index=False)
        if mod=='valid':
            cm=conf_mat(tot_label.cpu(),tot_pred.cpu())
            cm.savefig(f'./saved/{project_name}/{date}/{now}/checkpoint/checkpoint_{str(e).zfill(3)}.jpg')
            wandb.log({f'{str(e).zfill(3)}_confusion_matrix':wandb.Image(cm),'loss' : epoch_loss, 'accuracy' : epoch_acc,'f1 score':epoch_f1})
            plt.close()
            if best_f1<epoch_f1:
                stopper=0
                best_epoch=e
                best_f1=epoch_f1
                torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{dirs['checkpoint']}/{str(e).zfill(3)}_checkpoint_best_{epoch_f1:.5f}_{epoch_loss:.3f}_{epoch_acc:.3f}.pt")
            elif (prev_f1<epoch_f1):
                torch.save({'epoch':e,'loss':loss,'model_state_dict':model.state_dict(),'optimizer_state_dict':optim.state_dict()},f"{dirs['checkpoint']}/{str(e).zfill(3)}_checkpoint_{epoch_acc:.5f}_{epoch_f1:.3f}_{epoch_loss:.3f}.pt")
                stopper+=1
            else:
                stopper+=1
        elif mod=='test' and stopper==15:
            break
    if mod=='test' and stopper==15:
            break
wandb.finish()

wandb: wandb version 0.12.1 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade

CondaEnvException: Unable to determine environment

Please re-run this command with one of the following options:

* Provide an environment name via --name or -n
* Re-run this command inside an activated conda environment.



mod: valid



현재 epoch-0의 평균 Loss : 2.003, 평균 Accuracy : 0.983, F1 score : 0.983
mod: test



mod: train


RuntimeError: CUDA out of memory. Tried to allocate 138.00 MiB (GPU 0; 31.75 GiB total capacity; 6.49 GiB already allocated; 83.50 MiB free; 7.01 GiB reserved in total by PyTorch)

In [ ]:
for i in dirs:
    print(dirs[i])

In [ ]:
print(mod)

In [ ]:
print(pred)

In [ ]:
if mod=='valid':
        print(1)

In [ ]:
cm=conf_mat(cm_label.cpu(),cm_pred.cpu())

In [15]:
cm.savefig(f'./saved/{project_name}/{date}/{now}/checkpoint/checkpoint_{str(e).zfill(3)}.jpg')